In [1]:
import json
import pandas as pd
import requests
from simple_salesforce import Salesforce, SalesforceLogin, SFType

KeyboardInterrupt: 

In [ ]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

In [ ]:
results = sf.query_all(
    """
   select Id, IdentificationId__pc from Account 
    """
)
df = pd.DataFrame(results["records"])
df.shape
Pacientes_SF = df[["Id", "IdentificationId__pc"]]
Pacientes_SF.shape

(1540194, 2)

In [ ]:
# Recuperar empresas ID de salesforce
results = sf.query_all(
    """
   select id,Name,Nombre_Empresa__c,Empresa__c, Empresa_Padre__c from Programas_por_Empresas__c
    """
)
df = pd.DataFrame(results["records"])
df.shape
Empresas_SF = df[["Id", "Nombre_Empresa__c", "Name", "Empresa__c","Empresa_Padre__c"]]
Empresas_SF.head()



,Id,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,a1W8c000007vDN8EAM,Metlife Holding Metlife Laboratorio Maver Ltda.,A-0000,0018c00002axQXrAAM,Metlife
1,a1W8c000007vDN9EAM,Metlife Holding Metlife Aguas Del Valle S.A.,A-0001,0018c00002axQXsAAM,Metlife
2,a1W8c000007vDNAEA2,Metlife Holding Metlife Berliam Spa.,A-0002,0018c00002axQXtAAM,Metlife
3,a1W8c000007vDNBEA2,Metlife Holding Metlife Besalco Maquinarias S.A.,A-0003,0018c00002axQXuAAM,Metlife
4,a1W8c000007vDNCEA2,Metlife Holding Metlife Carlos Escarate Y Cia ...,A-0004,0018c00002axQXvAAM,Metlife


In [ ]:
Empresas_SF.shape

(7880, 5)

In [ ]:

# Relacion pacientes y empresas
results = sf.query_all(
    """
  select Id,Paciente__c,Programas_por_Empresas__c from Paciente_Programas__c 
    """
)
df = pd.DataFrame(results)
df.shape
Pacientes_empresas = pd.DataFrame(results["records"])
Pacientes_empresas = Pacientes_empresas[
    ["Paciente__c", "Programas_por_Empresas__c"]
]
Pacientes_empresas.head()

,Paciente__c,Programas_por_Empresas__c
0,0018c00002amLbpAAE,a1W8c000007vDPvEAM
1,0018c00002amLbqAAE,a1W8c000007vDSpEAM
2,0018c00002amLbrAAE,a1W8c000007vDSpEAM
3,0018c00002amLbsAAE,a1W8c000007vDSpEAM
4,0018c00002amLc2AAE,a1W8c000007vDSpEAM


In [ ]:
#une Pacientes y el que hace de union Pacientes_empresas
pacientes_y_union = pd.merge(
    left=Pacientes_SF,
    right=Pacientes_empresas,
    how="left",
    left_on="Id",
    right_on="Paciente__c",
)
pacientes_y_union.head()

,Id,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c
0,0018c00002amLTMAA2,173805004,0018c00002amLTMAA2,a1W8c0000083icPEAQ
1,0018c00002amLUGAA2,153338949,NaN,NaN
2,0018c00002amLUHAA2,177735213,0018c00002amLUHAA2,a1W8c0000083j8wEAA
3,0018c00002amLUIAA2,169332924,0018c00002amLUIAA2,a1W8c0000083j8uEAA
4,0018c00002amLUJAA2,172957692,0018c00002amLUJAA2,a1W8c0000083iWhEAI


In [ ]:
pacientes_y_union.shape

(1647766, 4)

In [ ]:
pacientes_y_empresas_final = pd.merge(
    left=pacientes_y_union,
    right=Empresas_SF,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id",
)
pacientes_y_empresas_final.head()

,Id_x,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c,Id_y,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,0018c00002amLTMAA2,173805004,0018c00002amLTMAA2,a1W8c0000083icPEAQ,a1W8c0000083icPEAQ,Metlife Vulco Vulco S.A.,A-1075,0018c00002cXJZeAAO,Metlife
1,0018c00002amLUGAA2,153338949,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0018c00002amLUHAA2,177735213,0018c00002amLUHAA2,a1W8c0000083j8wEAA,a1W8c0000083j8wEAA,Farmacias Cruz Verde Holding Cruz Verde Farmac...,A-2322,0018c00002cXclVAAS,Farmacias Cruz Verde
3,0018c00002amLUIAA2,169332924,0018c00002amLUIAA2,a1W8c0000083j8uEAA,a1W8c0000083j8uEAA,None,A-2320,None,None
4,0018c00002amLUJAA2,172957692,0018c00002amLUJAA2,a1W8c0000083iWhEAI,a1W8c0000083iWhEAI,Minera Abra Holding Minera Abra Minera Abra,A-0721,0018c00002cXJTrAAO,Minera Abra


In [ ]:
nuevos = ("F:/Empresas SF/EY/activo.xlsx")
Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16592\826961705.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  Pacientes_Archivo = pd.read_excel(nuevos)


,nombres,apellidos,BillingCountry,rut1,nacimiento,sexo,cargo,division,Identification_Type__c,email,rut
0,MERDAS,MAKRAM,Chile,250450354,1992-09-18,Hombre,Senior,FSO,RUT,NaN,250450354
1,José Alexander,Abreu Sandoval,Chile,267051372,1997-02-27,Hombre,Staff,TAX,RUT,NaN,267051372
2,MARÍA JOSÉ,ABUD HERRERA,Chile,195394474,1996-09-04,Mujer,Associate,Consulting,RUT,Maria.Jose.Abud@cl.ey.com,195394474
3,Santiago Tomás Alvarez,Alvarez,Chile,243922225,1982-01-10,Hombre,Manager,FSO,RUT,NaN,243922225
4,Nicolaz De Pablo,Araya Brito,Chile,17017450K,1988-05-15,Hombre,Staff,TAX,RUT,NaN,17017450K


In [ ]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.upper()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].astype(str)
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.strip()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["IdentificationId__pc"])

In [ ]:
Pacientes_Archivo.head()

,nombres,apellidos,BillingCountry,rut1,nacimiento,sexo,cargo,division,Identification_Type__c,email,rut,RutEstaoNo
0,Merdas,Makram,Chile,250450354,1992-09-18,Hombre,Senior,FSO,RUT,NaN,250450354,True
1,José Alexander,Abreu Sandoval,Chile,267051372,1997-02-27,Hombre,Staff,TAX,RUT,NaN,267051372,True
2,María José,Abud Herrera,Chile,195394474,1996-09-04,Mujer,Associate,Consulting,RUT,Maria.Jose.Abud@cl.ey.com,195394474,True
3,Santiago Tomás Alvarez,Alvarez,Chile,243922225,1982-01-10,Hombre,Manager,FSO,RUT,NaN,243922225,True
4,Nicolaz De Pablo,Araya Brito,Chile,17017450K,1988-05-15,Hombre,Staff,TAX,RUT,NaN,17017450K,True


In [ ]:
Pacientes_con_ID = pd.merge(
    left=Pacientes_Archivo,
    right=pacientes_y_empresas_final,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc",
)
Pacientes_con_ID.head()


,nombres,apellidos,BillingCountry,rut1,nacimiento,sexo,cargo,division,Identification_Type__c,email,...,RutEstaoNo,Id_x,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c,Id_y,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,Merdas,Makram,Chile,250450354,1992-09-18,Hombre,Senior,FSO,RUT,NaN,...,True,0018c00002ei2RcAAI,250450354,0018c00002ei2RcAAI,a1W8c0000083j8uEAA,a1W8c0000083j8uEAA,None,A-2320,None,None
1,José Alexander,Abreu Sandoval,Chile,267051372,1997-02-27,Hombre,Staff,TAX,RUT,NaN,...,True,0018c00002egntUAAQ,267051372,0018c00002egntUAAQ,a1W8c0000083j8uEAA,a1W8c0000083j8uEAA,None,A-2320,None,None
2,María José,Abud Herrera,Chile,195394474,1996-09-04,Mujer,Associate,Consulting,RUT,Maria.Jose.Abud@cl.ey.com,...,True,0018c00002f0Y3EAAU,195394474,0018c00002f0Y3EAAU,a1W8c0000083j8uEAA,a1W8c0000083j8uEAA,None,A-2320,None,None
3,Santiago Tomás Alvarez,Alvarez,Chile,243922225,1982-01-10,Hombre,Manager,FSO,RUT,NaN,...,True,0018c00002egntCAAQ,243922225,0018c00002egntCAAQ,a1W8c0000083j8uEAA,a1W8c0000083j8uEAA,None,A-2320,None,None
4,Nicolaz De Pablo,Araya Brito,Chile,17017450K,1988-05-15,Hombre,Staff,TAX,RUT,NaN,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
Pacientes_con_ID.to_excel(
    "F:/Empresas SF/EY/Pacientes_y_empresas_nuevas.xlsx", index=False
)


In [ ]:
Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]
Final_pacientes_y_empresa = Final_pacientes_y_empresa[
    Final_pacientes_y_empresa["RutEstaoNo"] == False
]

if ("nombres" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["FirstName"] = Final_pacientes_y_empresa["nombres"]
if ("apellidos" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["LastName"] = Final_pacientes_y_empresa["apellidos"]
if ("rut" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Final_pacientes_y_empresa["rut"]
if ("email" in Final_pacientes_y_empresa.columns) == True:
    Final_pacientes_y_empresa["email"] = Final_pacientes_y_empresa["email"].str.lower()
    Subir_pacientes["PersonEmail"] = Final_pacientes_y_empresa["email"]
if ("nacimiento" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Final_pacientes_y_empresa["nacimiento"]
if ("sexo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Final_pacientes_y_empresa["sexo"]
if ("direccion" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
if ("cargo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonTitle"] = Final_pacientes_y_empresa["cargo"]
if ("division" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonDepartment"] = Final_pacientes_y_empresa["division"]

Subir_pacientes["BillingCountry"] = "Chile"
Subir_pacientes["Identification_Type__c"] = "RUT"
Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"


Subir_pacientes.to_excel(
     "F:/Empresas SF/EY/account.xlsx", index=False
)
Subir_pacientes.head()


,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,PersonTitle,PersonDepartment,BillingCountry,Identification_Type__c,RecordTypeId
